Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Tensorflow logistic regression

In [5]:
learning_rate = 0.01
beta = 0.01

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes
tf_train_dataset = tf.constant(train_dataset)
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b)

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1)) + beta*tf.nn.l2_loss(W)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, W) + b)
valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W) + b)
test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, W) + b)

Let's run logistic regression

In [6]:
training_epochs = 25
batch_size = 1000
display_step = 1

with tf.Session() as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for epoch in range(training_epochs):
    total_batch = int(train_dataset.shape[0]/batch_size)
    # Loop over all batches
    for i in range(total_batch):
      # Generate a minibatch.
      batch_xs = train_dataset[i*batch_size:(i+1)*batch_size, :] 
      batch_ys = train_labels[i*batch_size:(i+1)*batch_size, :]
      # Run optimization
      _, c = session.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})

    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
      print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
      print("LR Training accuracy: %.1f%%" % accuracy(train_prediction.eval(), train_labels))
      print("LR Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Epoch: 0001 cost= 1.008811474
LR Training accuracy: 78.5%
LR Validation accuracy: 78.6%
Epoch: 0002 cost= 0.884338915
LR Training accuracy: 79.7%
LR Validation accuracy: 80.1%
Epoch: 0003 cost= 0.839709759
LR Training accuracy: 80.4%
LR Validation accuracy: 80.7%
Epoch: 0004 cost= 0.816603005
LR Training accuracy: 80.8%
LR Validation accuracy: 81.1%
Epoch: 0005 cost= 0.802380443
LR Training accuracy: 81.1%
LR Validation accuracy: 81.3%
Epoch: 0006 cost= 0.792684853
LR Training accuracy: 81.3%
LR Validation accuracy: 81.5%
Epoch: 0007 cost= 0.785617113
LR Training accuracy: 81.5%
LR Validation accuracy: 81.7%
Epoch: 0008 cost= 0.780218184
LR Training accuracy: 81.6%
LR Validation accuracy: 81.8%
Epoch: 0009 cost= 0.775948942
LR Training accuracy: 81.7%
LR Validation accuracy: 81.9%
Epoch: 0010 cost= 0.772481918
LR Training accuracy: 81.8%
LR Validation accuracy: 82.0%
Epoch: 0011 cost= 0.769606590
LR Training accuracy: 81.9%
LR Validation accuracy: 82.1%
Epoch: 0012 cost= 0.

Tensorflow NN

In [7]:
batch_size = 128
beta = 0.01

def predict_with_hidden(dataset, weights1, biases1, weights2, biases2):
  return tf.matmul(tf.nn.relu(tf.matmul(dataset, weights1) + biases1), weights2) + biases2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = predict_with_hidden(tf_train_dataset, weights1, biases1, weights2, biases2)
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    ) + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    predict_with_hidden(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(predict_with_hidden(tf_test_dataset, weights1, biases1, weights2, biases2))

Let's run NN

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3532.714111
Minibatch accuracy: 7.0%
Validation accuracy: 31.7%
Minibatch loss at step 500: 21.257662
Minibatch accuracy: 83.6%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 1.025557
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.769532
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.560865
Minibatch accuracy: 89.8%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.625160
Minibatch accuracy: 86.7%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.868381
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Test accuracy: 90.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 5001

sample_train_dataset = train_dataset[:500, :]
sample_train_labels = train_labels[:500, :]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (sample_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = sample_train_dataset[offset:(offset + batch_size), :]
    batch_labels = sample_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3427.204102
Minibatch accuracy: 10.2%
Validation accuracy: 43.9%
Minibatch loss at step 500: 21.023222
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 0.490288
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.296253
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 2000: 0.282688
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.275832
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.274242
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 3500: 0.276674
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 4000: 0.276982
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 4500: 0.270464
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 5000: 0.265913
Minibatch accuracy: 100.0%
Vali

Restrict training data to 500, results depict an overfitting on training data, with accuracy on training data increasing to 100% while droping on both valid and test data.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128
beta = 0.01

def predict_with_dropout(dataset, weights1, biases1, weights2, biases2, keep_prob):
  return tf.matmul(
    tf.nn.dropout(
      tf.nn.relu(tf.matmul(dataset, weights1) + biases1), keep_prob), weights2) + biases2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  keep_prob = tf.placeholder(tf.float32)
  
  # Training computation.
  logits = predict_with_dropout(tf_train_dataset, weights1, biases1, weights2, biases2, keep_prob)
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    ) + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    predict_with_dropout(tf_valid_dataset, weights1, biases1, weights2, biases2, keep_prob))
  test_prediction = tf.nn.softmax(
    predict_with_dropout(tf_test_dataset, weights1, biases1, weights2, biases2, keep_prob))

In [11]:
num_steps = 5001

sample_train_dataset = train_dataset[:500, :]
sample_train_labels = train_labels[:500, :]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (sample_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = sample_train_dataset[offset:(offset + batch_size), :]
    batch_labels = sample_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3558.818604
Minibatch accuracy: 13.3%
Validation accuracy: 33.7%
Minibatch loss at step 500: 21.174177
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 0.529346
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 0.325641
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.320661
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.302236
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 0.299860
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3500: 0.304968
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 4000: 0.300915
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 4500: 0.311114
Minibatch accuracy: 99.2%
Validation accuracy: 78.5%
Minibatch loss at step 5000: 0.286250
Minibatch accuracy: 100.0%
Valid

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


learning rate decay

In [12]:
batch_size = 128
beta = 0.01

def predict_with_dropout(dataset, weights1, biases1, weights2, biases2, keep_prob):
  return tf.matmul(
    tf.nn.dropout(
      tf.nn.relu(tf.matmul(dataset, weights1) + biases1), keep_prob), weights2) + biases2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  keep_prob = tf.placeholder(tf.float32)

  # Decay learning rate.
  global_step = tf.Variable(0)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.8)
  
  # Training computation.
  logits = predict_with_dropout(tf_train_dataset, weights1, biases1, weights2, biases2, keep_prob)
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    ) + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    predict_with_dropout(tf_valid_dataset, weights1, biases1, weights2, biases2, keep_prob))
  test_prediction = tf.nn.softmax(
    predict_with_dropout(tf_test_dataset, weights1, biases1, weights2, biases2, keep_prob))

In [13]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3612.699463
Minibatch accuracy: 9.4%
Validation accuracy: 25.0%
Minibatch loss at step 500: 151.965195
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 56.811165
Minibatch accuracy: 78.9%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 41.084522
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 36.955162
Minibatch accuracy: 89.1%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 35.729164
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 35.494759
Minibatch accuracy: 80.5%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 35.325394
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 4000: 35.259926
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Minibatch loss at step 4500: 35.140572
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Minibatch loss at step 5000: 35.227180
Minibatch accuracy: 81.2%
Valid

NN structure (784,1024,300,100,10)

In [14]:
batch_size = 1024
beta = 0.005

def multilayer_with_dropout(dataset, weights, biases, keep_prob):
  weights_index = sorted(weights.keys())
  biases_index = sorted(biases.keys())
  hidden_layer = dataset
  for i in xrange(len(weights_index)-1):
    hidden_layer = tf.matmul(hidden_layer, weights[weights_index[i]]) + biases[biases_index[i]]
    hidden_layer = tf.nn.dropout(tf.nn.relu(hidden_layer), keep_prob)
  # Output layer
  return tf.matmul(hidden_layer, weights[weights_index[-1]]) + biases[biases_index[-1]]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1': tf.get_variable('h1', shape=[image_size * image_size, 1024], initializer=tf.contrib.layers.xavier_initializer()),
    'h2': tf.get_variable('h2', shape=[1024, 300], initializer=tf.contrib.layers.xavier_initializer()),
    'h3': tf.get_variable('h3', shape=[300, 100], initializer=tf.contrib.layers.xavier_initializer()),
    'h4': tf.get_variable('h4', shape=[100, num_labels], initializer=tf.contrib.layers.xavier_initializer())
  }
  biases = {
    'b1': tf.Variable(tf.zeros([1024])),
    'b2': tf.Variable(tf.zeros([300])),
    'b3': tf.Variable(tf.zeros([100])),
    'b4': tf.Variable(tf.zeros([num_labels]))
  }
  keep_prob = tf.placeholder(tf.float32)

  # Decay learning rate.
  global_step = tf.Variable(0)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.8)
  
  # Training computation.
  logits = multilayer_with_dropout(
        tf_train_dataset, weights, biases, keep_prob)
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    ) + beta * sum([tf.nn.l2_loss(weight) for weight in weights.values()])
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    multilayer_with_dropout(tf_valid_dataset, weights, biases, keep_prob))
  test_prediction = tf.nn.softmax(
    multilayer_with_dropout(tf_test_dataset, weights, biases, keep_prob))

In [15]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 6.439913
Minibatch accuracy: 11.1%
Validation accuracy: 28.8%
Minibatch loss at step 500: 1.546499
Minibatch accuracy: 83.7%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 1.130589
Minibatch accuracy: 86.0%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 1.098950
Minibatch accuracy: 85.3%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 1.090904
Minibatch accuracy: 85.4%
Validation accuracy: 86.3%
Minibatch loss at step 2500: 1.072868
Minibatch accuracy: 85.5%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 1.036531
Minibatch accuracy: 85.7%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 1.063382
Minibatch accuracy: 84.5%
Validation accuracy: 86.3%
Minibatch loss at step 4000: 1.072132
Minibatch accuracy: 85.1%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 1.135489
Minibatch accuracy: 83.0%
Validation accuracy: 86.3%
Minibatch loss at step 5000: 1.034248
Minibatch accuracy: 86.3%
Validation accurac